In [106]:
import torch
import nltk
from nltk.corpus import stopwords
import warnings
import pickle
import GloVe
warnings.simplefilter("ignore")

import tensorflow as tf
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.layers import Dense, Embedding, Input, Add, Dot, Reshape, Flatten
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.sequence import skipgrams
from tensorflow.python.keras.models import Model, load_model

import tarfile
from urllib.request import urlretrieve
import os
import nltk
from scipy.sparse import csr_matrix, lil_matrix
import numpy as np
from scipy.sparse import save_npz, load_npz
from tensorflow.python.keras import backend as K
import random
import matplotlib.pyplot as plt
from tensorflow.python.keras.utils import plot_model
import pandas as pd
%matplotlib inline

In [28]:
f = open('mySavedDict.txt', 'rb')
data = pickle.load(f)
f.close()

In [56]:
contexts = []
questions = []
answers = []
impossible = []
titles = []
for title, paragraphs in data.items():
    titles.append(title)
    for key, value in paragraphs.items():
        if key == 'context':
            contexts.append(value)
        else:
            questions.append(key)
            answers.append(value['answers'])
            impossible.append(value['impossible'])

            
            

In [57]:
len(questions) - len(answers) # each question has merely one answer

0

In [76]:
contextWords = [[context[i].split()] for i in range(len(context))]
questionWords = [[questions[i].split()] for i in range(len(questions))]
answerWords = []
for i in range(len(answers)):
    if len(answers[i]) > 0:
        current = answers[i][0].split()
    else:
        current = ""
    answerWords.append(current)
        
        
    

In [107]:
stop_words = stopwords.words('english')

In [115]:
questions1 = []
for question in questions:
    ques1 = []
    ques = question.split()
    for word in ques:
        if word not in stop_words:
            ques1.append(word)
    questions1.append(ques1)
    
answers1 = []
for answer in answers:
    ques1 = []
    ques = question.split()
    for word in ques:
        if word not in stop_words:
            ques1.append(word)
    questions1.append(ques1)
            



In [112]:
questions[0].split()

['When', 'did', 'Beyonce', 'start', 'becoming', 'popular?']